# Hashについて色々考えてみた

## 元の文字列がめっちゃ近いのにハッシュ値が同じになるという話
- 2024/03/20 辺りに以下のツイートが話題になった
  - https://x.com/realhashbreaker/status/1770161965006008570

```text
Here is a 72-byte alphanum MD5 collision with 1-byte difference for fun:
 md5("TEXTCOLLBYfGiJUETHQ4hAcKSMd5zYpgqf1YRDhkmxHkhPWptrkoyz28wnI9V0aHeAuaKnak")
=
md5("TEXTCOLLBYfGiJUETHQ4hEcKSMd5zYpgqf1YRDhkmxHkhPWptrkoyz28wnI9V0aHeAuaKnak")
```

> これは、1 バイトの違いがある 72 バイトの英数字 MD5 衝突です。


### つまりどういうこと？
- わからないので試してみた

In [1]:
import hashlib
import bcrypt # pip install bcrypt

In [2]:
text1 = "TEXTCOLLBYfGiJUETHQ4hAcKSMd5zYpgqf1YRDhkmxHkhPWptrkoyz28wnI9V0aHeAuaKnak"
text2 = "TEXTCOLLBYfGiJUETHQ4hEcKSMd5zYpgqf1YRDhkmxHkhPWptrkoyz28wnI9V0aHeAuaKnak"
#                             ↑22文字目が「A」と「E」で異なる
assert text1 != text2

In [3]:
hash1 = hashlib.md5(text1.encode()).hexdigest()
hash2 = hashlib.md5(text2.encode()).hexdigest()

assert hash1 == hash2
print(hash1, hash2)


faad49866e9498fc1719f5289e7a0269 faad49866e9498fc1719f5289e7a0269


### つまり？
- md5で上記の2つの文字列をハッシュ化すると同じハッシュ値が出力される
- 元の2つの文字列が1字しか違わない
- 72文字しかない

↑っというところが凄い。

### 脆弱性とかそういう話じゃないの？
- そうではない
- そもそもmd5は古いハッシュ関数であり、現在では使われるべきではない
- よく考えてみると md5は128bitのハッシュ値 -> 文字列化すると32文字 → 72文字の文字列をハッシュ化するのだからハッシュ値が衝突するに決まっている
  - 数が違うのだからすべてユニークになるわけがない
  - →32文字で表される ものと 72文字で表されるもの は表される値が違う
- ただ、1文字しか違わないところが「面白い」というだけの話

## ハッシュが重複しても良いってどういうことか考えてみた
- ハッシュは重複する可能性がある
- 可能性はあるが、実際に重複する確率は極めて低いので無視できる
  - 例えばGitのコミットハッシュは40桁ある
    - "非常に大きなプロジェクトで、1000人のアクティブな開発者が、日に10回のコミットをしたとする。この場合、4×10の17乗年間活動を続けると、50%の確率で衝突が発生する。"
      - （宇宙の年齢よりも長い）
    - https://dassencio.org/20

### それはそうなんだけどログインパスワードとかいいんだっけ？
- そもそも何が問題なんだっけ？
- パスワード「a」とパスワード「b」のハッシュが同じだった時に何が問題あるのだろう？
- 事象「ログイン時にパスワード入力する時に「a」と入れても「b」と入れてもログインできてしまう」
- それはよくないことだと思うけど、なぜよくないのか？
- 不正ログインされる場合にランダムに入力される際にログインできる確率が2倍になる
- 例えば実際のパスワードは超長くて複雑な文字列だけど、ハッシュが同じだったら「a」とか「b」とか短い文字列でログインできるかもしれない
- それはそうなんだけど、なぜ許されているのか。

### もうちょっと具体的にしてみる
- あるサービスでログインパスワードとして「超長くて複雑な文字列」を設定したとする
- そのサービスが攻撃されて、不正ログインを狙うためにランダムにパスワードを入力する攻撃が行われたとする
- 攻撃者はよくあるパスワードの一覧を持っており、よくあるパスワードを試す
- よくあるパスワードのハッシュと「超長くて複雑な文字列」のハッシュが同じだった場合、攻撃者はよくあるパスワードを利用して本来は「超長くて複雑な文字列」を入力しないとログインできないはずのサービスにログインできてしまう

### この懸念は理論上は正しいが、実際には問題にならない
- 事実上、ハッシュが重複することはほぼないから。
  - 結局この1点らしい。。。

## ソルトとかペッパーとかストレッチングとか聞いたことあるけどその辺りはどうなん？
- ソルトやペッパー、ストレッチングなどを使っても結局ハッシュは衝突する
- これらは全てパスワードハッシュが漏洩した際の防御策である

ここから、ソルトとかペッパーとかストレッチとかを試してみます

- 前提
  - データベースを使用してユーザーの認証を行う
  - データは漏洩する
  - ユーザーはパスワードを使いまわす


In [7]:
# 一番シンプルなデータ保存の形
# ユーザー名とパスワードをそのまま保存する
"""
| user_name | password |
|-----------|----------|
| user1     | 12345    |
| user2     | password |
| user3     | 12345    |
"""

# メリット
"""
- 実装が容易
"""

# デメリット
"""
- データベースが侵害された場合、パスワードがそのまま漏洩する
- 漏洩したサービスのアカウントが不正ログインされる
- 更に他のサービスもアカウントが不正ログインされる
  - ユーザーはパスワードを使いまわす
"""

'\n- データベースが侵害された場合、パスワードがそのまま漏洩する\n- 漏洩したサービスのアカウントが不正ログインされる\n- 更に他のサービスもアカウントが不正ログインされる\n  - ユーザーはパスワードを使いまわす\n'

In [8]:
import hashlib
# パスワードのハッシュ化
# パスワードをハッシュ化して保存する
# → 入力されたパスワードを同じようにハッシュ化して、保存されているハッシュ値と比較する

hash11 = hashlib.sha256("12345".encode()).hexdigest()
hash12 = hashlib.sha256("password".encode()).hexdigest()
print(hash11, hash12)

"""
| user_name | password_hash                                                    |
|-----------|------------------------------------------------------------------|
| user1     | 5994471abb01112afcc18159f6cc74b4f511b99806da59b3caf5a9c173cacfc5 |
| user2     | 5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8 |
| user3     | 5994471abb01112afcc18159f6cc74b4f511b99806da59b3caf5a9c173cacfc5 |
"""

# メリット
"""
- データベースが侵害された場合でも、パスワードを知ることができない
"""

# デメリット
"""
- 同じパスワードのハッシュ値が同じになる
- レインボーテーブル攻撃に弱い
  - あらかじめ計算しておいたハッシュ値とパスワードの対応表を使って、ハッシュ値から元のパスワードを逆引きする攻撃
  - ハッシュ値が同じならパスワードも同じ
"""

5994471abb01112afcc18159f6cc74b4f511b99806da59b3caf5a9c173cacfc5 5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8


'\n- 同じパスワードのハッシュ値が同じになる\n- レインボーテーブル攻撃に弱い\n  - あらかじめ計算しておいたハッシュ値とパスワードの対応表を使って、ハッシュ値から元のパスワードを逆引きする攻撃\n  - ハッシュ値が同じならパスワードも同じ\n'

In [9]:
# ソルトを使ったハッシュ化
# パスワードにソルトを付けてハッシュ化する（ユーザー毎に異なるソルトを使う）
"""
| user_name | password_hash                    | salt |
|-----------|----------------------------------|------|
| user1     | df6a0e03d56c152b90ef2671bc1a6de6a0b300668b27c44e028a2a2e86bb0ea8 | 1234 |
| user2     | 744a92b96a293fe9af00134b37a2e9ad72be7162f7f3167b2c3ee0419f1d682b | 5678 |
| user3     | 70f33726b270155dde08000f2b5ecd73e73844f15259a50e43e7b499720a417f | abcd |
"""
hash21 = hashlib.sha256(("12345" + "1234").encode()).hexdigest()
hash22 = hashlib.sha256(("password" + "5678").encode()).hexdigest()
hash23 = hashlib.sha256(("12345" + "abcd").encode()).hexdigest()
print(hash21, hash22, hash23)

# メリット
"""
- 同じパスワードでも、異なるハッシュ値になる
"""

# デメリット
"""
- 管理上のコストが増える
  - ソルトの生成、保存が必要
- データベースが侵害された場合、ハッシュとソルトを知ることができるため、時間をかければパスワードを知ることができる
"""

df6a0e03d56c152b90ef2671bc1a6de6a0b300668b27c44e028a2a2e86bb0ea8 744a92b96a293fe9af00134b37a2e9ad72be7162f7f3167b2c3ee0419f1d682b 70f33726b270155dde08000f2b5ecd73e73844f15259a50e43e7b499720a417f


'\n- 管理上のコストが増える\n  - ソルトの生成、保存が必要\n- データベースが侵害された場合、ハッシュとソルトを知ることができるため、時間をかければパスワードを知ることができる\n'

In [10]:
# ソルト + ぺッパーを使ったハッシュ化
# ソルトに加えて、固定の文字列（ぺッパー）を付けてハッシュ化する
"""
| user_name | password_hash                                                    | salt |
|-----------|------------------------------------------------------------------|------|
| user1     | 61b83a68bda7b6c17ce6f3f79f2492ae59be01da5024e2aa8663f18db7574faf | 1234 |
| user2     | 0962408c9b8e1920305e1eeb6bbefaf6bfb933967c9322d9193c9abf16188bac | 5678 |
| user3     | abd1edd84e6911fcffc034c5747cd7f21ed955642871356f3ee0c76227fdb061 | abcd |
"""
pepper = "xyz"
hash31 = hashlib.sha256(("12345" + "1234" + pepper).encode()).hexdigest()
hash32 = hashlib.sha256(("password" + "5678" + pepper).encode()).hexdigest()
hash33 = hashlib.sha256(("12345" + "abcd" + pepper).encode()).hexdigest()
print(hash31, hash32, hash33)

# メリット
"""
- ソルトのみの場合よりも、より安全なハッシュ化が可能
"""

# デメリット
"""
- 管理上のコストが増える
  - ソルトの生成、保存が必要
- ぺッパーの管理が必要
  - ペッパーはアプリケーションコードや安全な環境設定ファイルなど、データベース外部の安全な場所に保管する必要があります
- ペッパーが漏洩した場合、ペッパーをバージョン管理する必要がある
  - すると、使用されたペッパーを特定するには？っという情報が必要になり複雑化する
"""

61b83a68bda7b6c17ce6f3f79f2492ae59be01da5024e2aa8663f18db7574faf 0962408c9b8e1920305e1eeb6bbefaf6bfb933967c9322d9193c9abf16188bac abd1edd84e6911fcffc034c5747cd7f21ed955642871356f3ee0c76227fdb061


'\n- 管理上のコストが増える\n  - ソルトの生成、保存が必要\n- ぺッパーの管理が必要\n  - ペッパーはアプリケーションコードや安全な環境設定ファイルなど、データベース外部の安全な場所に保管する必要があります\n- ペッパーが漏洩した場合、ペッパーをバージョン管理する必要がある\n  - すると、使用されたペッパーを特定するには？っという情報が必要になり複雑化する\n'

In [11]:
# ソルト + ストレッチングを使ったハッシュ化
# ハッシュ化した結果を何度もハッシュ化する

"""
| user_name | password_hash                                                    | salt |
|-----------|------------------------------------------------------------------|------|
| user1     | 06fb055eeaad491dba955d7e6ed010e0f3d9968e1a255965af061a8079c2ce9f | 1234 |
| user2     | 30a9db8394400eb6f9238df60b70fd7dabb4023f4e261055b5b187d30cd0446e | 5678 |
| user3     | 8af4bdbb493a7c2b6d43ec0178098fef221c072f72a45fe6cfeb4b507eae7e2d | abcd |
"""

hash41_1 = hashlib.sha256(("12345" + "1234").encode()).hexdigest()
hash41_2 = hashlib.sha256(hash41_1.encode()).hexdigest()
hash42_1 = hashlib.sha256(("password" + "5678").encode()).hexdigest()
hash42_2 = hashlib.sha256(hash42_1.encode()).hexdigest()
hash43_1 = hashlib.sha256(("12345" + "abcd").encode()).hexdigest()
hash43_2 = hashlib.sha256(hash43_1.encode()).hexdigest()
print(hash41_2, hash42_2, hash43_2)

# メリット
"""
- 同じパスワードでも、異なるハッシュ値になる
- 生成ロジック + ハッシュ + saltが漏洩したとしても、パスワードを解析するには時間がかかる
"""

# デメリット
"""
- 管理上のコストが増える
  - ソルトの生成、保存が必要
- ☆ハッシュ化の計算コストが増える
"""

"""
- 「ハッシュ化の計算コストが増える」はデメリットとして挙げられるが、ブルートフォース攻撃に対して有効な手法である
  - パスワードを総当たりで解析する攻撃
"""

06fb055eeaad491dba955d7e6ed010e0f3d9968e1a255965af061a8079c2ce9f 30a9db8394400eb6f9238df60b70fd7dabb4023f4e261055b5b187d30cd0446e 8af4bdbb493a7c2b6d43ec0178098fef221c072f72a45fe6cfeb4b507eae7e2d


'\n- 「ハッシュ化の計算コストが増える」はデメリットとして挙げられるが、ブルートフォース攻撃に対して有効な手法である\n  - パスワードを総当たりで解析する攻撃\n'